# 🧭 SOLAQUA Navigation & Guidance System Analysis

**Comprehensive analysis of underwater vehicle navigation and guidance systems**

This notebook provides detailed analysis of:
- 🎯 Guidance system error components and performance
- 🧭 Navigation plane approximation data
- 📍 Position trajectory analysis
- ⚖️ Navigation vs guidance system comparison
- 📊 Performance assessment and recommendations

---

In [53]:
# 🔧 IMPORT REQUIRED LIBRARIES
# ============================

# Core data analysis
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from datetime import datetime

# Scientific computing
try:
    from scipy import interpolate
    HAS_SCIPY = True
except ImportError:
    HAS_SCIPY = False
    print("📦 SciPy not available - some correlation analysis features disabled")

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec

# Enhanced plotting (optional - graceful fallback if not available)
try:
    import seaborn as sns
    sns.set_style("whitegrid")
    HAS_SEABORN = True
except ImportError:
    HAS_SEABORN = False
    print("📦 Seaborn not available - using matplotlib defaults")

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.offline as pyo
    HAS_PLOTLY = True
except ImportError:
    HAS_PLOTLY = False
    print("📦 Plotly not available - using matplotlib only")

# Custom utilities
import utils.navigation_guidance_analysis as nav_guidance_utils

# Configuration
warnings.filterwarnings('ignore', category=FutureWarning)
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully!")
print(f"📊 Enhanced visualization: Seaborn={HAS_SEABORN}, Plotly={HAS_PLOTLY}, SciPy={HAS_SCIPY}")

✅ Libraries imported successfully!
📊 Enhanced visualization: Seaborn=True, Plotly=True, SciPy=True


In [54]:
# 🔍 CONFIGURATION PARAMETERS
# ===========================

# Data paths
BY_BAG_FOLDER = "exports/by_bag"
OUTPUT_FOLDER = "exports/outputs"

# Analysis configuration
BAG_SELECTION = "2024-08-20_13-40-35"  # Specific bag with complete dataset
SENSOR_SELECTION = None  # None = analyze all available sensors

# Visualization settings
FIGURE_SIZE = (15, 10)
PLOT_STYLE = "plotly"  # "plotly" or "matplotlib"
COLOR_PALETTE = "Set1"
EXPORT_PLOTS = False
EXPORT_SUMMARY = True

# Create output directory
Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

print("⚙️ Configuration loaded:")
print(f"   📁 Data folder: {BY_BAG_FOLDER}")
print(f"   📊 Output folder: {OUTPUT_FOLDER}")
print(f"   🎨 Plot style: {PLOT_STYLE}")
print(f"   📈 Export enabled: Plots={EXPORT_PLOTS}, Summary={EXPORT_SUMMARY}")

⚙️ Configuration loaded:
   📁 Data folder: exports/by_bag
   📊 Output folder: exports/outputs
   🎨 Plot style: plotly
   📈 Export enabled: Plots=False, Summary=True


In [55]:
# 🚀 INITIALIZE NAVIGATION & GUIDANCE ANALYZER
# ============================================

# Reload utility module to get latest updates
import importlib
importlib.reload(nav_guidance_utils)

# Initialize navigation/guidance analyzer
nav_analyzer = nav_guidance_utils.NavigationGuidanceAnalyzer(BY_BAG_FOLDER)

# Get first available bag for analysis
if BAG_SELECTION is None and nav_analyzer.available_bags:
    selected_bag = nav_analyzer.available_bags[0]
else:
    selected_bag = BAG_SELECTION

print(f"\n📊 Analysis Setup:")
print(f"   📅 Available bags: {len(nav_analyzer.available_bags)}")
print(f"   📊 Available sensors: {len(nav_analyzer.available_sensors)}")
print(f"   🎯 Selected bag: {selected_bag}")
print(f"   📋 Sensors: {', '.join(nav_analyzer.available_sensors)}")

🔍 Found Navigation/Guidance data:
   📅 Bags: 5
   📊 Sensors: 3
   Sensors: guidance, navigation_plane_approximation, navigation_plane_approximation_position

📊 Analysis Setup:
   📅 Available bags: 5
   📊 Available sensors: 3
   🎯 Selected bag: 2024-08-20_13-40-35
   📋 Sensors: guidance, navigation_plane_approximation, navigation_plane_approximation_position


## 🎯 Guidance System Error Analysis

Analysis of the guidance system's error components including:
- Position errors (X, Y, Z)
- Velocity errors (surge, sway, heave) 
- Attitude errors (yaw)
- Net distance tracking errors
- Desired vs actual performance comparison

In [56]:
# 🎯 GUIDANCE SYSTEM ERROR ANALYSIS (SCALE-CORRECTED)
# ====================================================

print("🎯 Guidance System Error Analysis")
print("=" * 40)

# Analyze guidance system errors
nav_analyzer.analyze_guidance_errors(selected_bag, interactive=HAS_PLOTLY)

# Load guidance data for detailed analysis
guidance_data = nav_analyzer.load_sensor_data("guidance", selected_bag)

if guidance_data is not None:
    print(f"\n📊 Guidance Data Summary:")
    print(f"   📈 Total samples: {len(guidance_data)}")
    print(f"   ⏱️ Duration: {guidance_data['t_rel'].max()/60:.2f} minutes")
    print(f"   📊 Sampling rate: {len(guidance_data)/(guidance_data['t_rel'].max()/60):.1f} samples/min")
    
    # 🔍 SCALE AND UNIT ANALYSIS
    print(f"\n🔍 SCALE & UNIT VERIFICATION:")
    print("-" * 35)
    
    # Identify position coordinates vs true errors
    position_error_cols = ['error_x', 'error_y', 'error_z']
    true_error_cols = ['error_net_distance', 'error_surge', 'error_sway', 'error_heave', 'error_depth']
    angular_error_cols = ['error_yaw']
    
    print(f"📍 POSITION VALUES (These are coordinates, NOT errors):")
    for col in position_error_cols:
        if col in guidance_data.columns:
            values = guidance_data[col]
            print(f"   • {col}: Range=[{values.min():.1f}, {values.max():.1f}] m")
            print(f"     Mean={values.mean():.1f} m, Std={values.std():.1f} m")
    
    # Calculate position magnitude for reference
    if all(col in guidance_data.columns for col in position_error_cols):
        pos_magnitude = np.sqrt(guidance_data['error_x']**2 + guidance_data['error_y']**2 + guidance_data['error_z']**2)
        print(f"   • 3D Position Magnitude: Range=[{pos_magnitude.min():.1f}, {pos_magnitude.max():.1f}] m")
        print(f"     (This represents distance from origin/reference frame)")
    
    print(f"\n🎯 TRUE TRACKING ERRORS (Properly scaled performance metrics):")
    for col in true_error_cols:
        if col in guidance_data.columns:
            values = guidance_data[col]
            abs_mean = values.abs().mean()
            abs_max = values.abs().max()
            rms = np.sqrt(np.mean(values**2))
            
            # Identify potential scale issues
            scale_issue = ""
            if abs_mean > 10:  # Suspiciously large for tracking errors
                scale_issue = " ⚠️ SCALE ISSUE?"
            elif abs_mean < 0.001:  # Suspiciously small
                scale_issue = " ⚠️ VERY SMALL"
            
            print(f"   • {col.replace('error_', '').title()}: Mean={abs_mean:.4f} m, Max={abs_max:.4f} m, RMS={rms:.4f} m{scale_issue}")
    
    print(f"\n🧭 ANGULAR ERRORS (Checking for radians vs degrees):")
    for col in angular_error_cols:
        if col in guidance_data.columns:
            values = guidance_data[col]
            abs_mean_rad = values.abs().mean()
            abs_max_rad = values.abs().max()
            
            # Convert to degrees for interpretation
            abs_mean_deg = np.degrees(abs_mean_rad)
            abs_max_deg = np.degrees(abs_max_rad)
            
            # Check if values make sense in radians or degrees
            if abs_max_rad > 2*np.pi:
                unit_interpretation = "❓ Possibly degrees (very large for radians)"
            elif abs_max_rad > np.pi:
                unit_interpretation = "⚠️ Large angles - check if radians or degrees"
            else:
                unit_interpretation = "✅ Likely radians (reasonable range)"
            
            print(f"   • {col}: Raw values: Mean={abs_mean_rad:.3f}, Max={abs_max_rad:.3f}")
            print(f"     As degrees: Mean={abs_mean_deg:.1f}°, Max={abs_max_deg:.1f}°")
            print(f"     Interpretation: {unit_interpretation}")
    
    # 📊 DESIRED VALUES ANALYSIS (Check for reference/setpoint scaling)
    desired_cols = [col for col in guidance_data.columns if col.startswith('desired_')]
    if desired_cols:
        print(f"\n📊 DESIRED/SETPOINT VALUES:")
        for col in desired_cols[:5]:  # Show first 5 desired values
            if col in guidance_data.columns:
                values = guidance_data[col]
                is_constant = values.std() < 1e-6
                
                if is_constant:
                    print(f"   • {col}: CONSTANT = {values.iloc[0]:.6f}")
                else:
                    print(f"   • {col}: Range=[{values.min():.3f}, {values.max():.3f}], Mean={values.mean():.3f}")
                
                # Special handling for known units
                if 'yaw' in col.lower():
                    if abs(values.iloc[0] - np.pi/2) < 0.01:
                        print(f"     (≈ π/2 radians = 90°)")
    
    # 🎯 PERFORMANCE ASSESSMENT WITH CORRECT SCALING
    print(f"\n🎯 CORRECTED PERFORMANCE METRICS:")
    print("-" * 35)
    
    # Position tracking (using net distance error - the true tracking metric)
    if 'error_net_distance' in guidance_data.columns:
        net_error = guidance_data['error_net_distance'].abs()
        print(f"📍 Position Tracking Performance:")
        print(f"   • Net distance error: Mean={net_error.mean():.3f} m, RMS={np.sqrt(np.mean(net_error**2)):.3f} m")
        print(f"   • 95th percentile: {np.percentile(net_error, 95):.3f} m")
        print(f"   • Maximum error: {net_error.max():.3f} m")
        
        # Performance rating for position tracking
        if net_error.mean() < 0.5:
            pos_rating = "🟢 Excellent"
        elif net_error.mean() < 1.0:
            pos_rating = "🟡 Good"
        elif net_error.mean() < 2.0:
            pos_rating = "🟠 Acceptable"
        else:
            pos_rating = "🔴 Needs improvement"
        
        print(f"   • Position tracking rating: {pos_rating}")
    
    # Velocity tracking performance
    velocity_errors = ['error_surge', 'error_sway', 'error_heave']
    velocity_performance = []
    
    print(f"\n🚀 Velocity Tracking Performance:")
    for vel_col in velocity_errors:
        if vel_col in guidance_data.columns:
            vel_error = guidance_data[vel_col].abs()
            vel_rms = np.sqrt(np.mean(guidance_data[vel_col]**2))
            velocity_performance.append(vel_error.mean())
            
            print(f"   • {vel_col.replace('error_', '').title()}: Mean={vel_error.mean():.4f} m/s, RMS={vel_rms:.4f} m/s")
    
    if velocity_performance:
        avg_vel_error = np.mean(velocity_performance)
        print(f"   • Average velocity error: {avg_vel_error:.4f} m/s")
        
        # Velocity performance rating
        if avg_vel_error < 0.1:
            vel_rating = "🟢 Excellent"
        elif avg_vel_error < 0.2:
            vel_rating = "🟡 Good"
        elif avg_vel_error < 0.5:
            vel_rating = "🟠 Acceptable"
        else:
            vel_rating = "🔴 Needs improvement"
        
        print(f"   • Velocity tracking rating: {vel_rating}")
    
    # Heading/yaw tracking performance
    if 'error_yaw' in guidance_data.columns:
        yaw_error_rad = guidance_data['error_yaw'].abs()
        yaw_error_deg = np.degrees(yaw_error_rad)
        
        print(f"\n🧭 Heading Tracking Performance:")
        print(f"   • Yaw error: Mean={yaw_error_deg.mean():.1f}°, RMS={np.degrees(np.sqrt(np.mean(guidance_data['error_yaw']**2))):.1f}°")
        print(f"   • 95th percentile: {np.percentile(yaw_error_deg, 95):.1f}°")
        
        # Heading performance rating
        if yaw_error_deg.mean() < 5:
            yaw_rating = "🟢 Excellent"
        elif yaw_error_deg.mean() < 10:
            yaw_rating = "🟡 Good"
        elif yaw_error_deg.mean() < 20:
            yaw_rating = "🟠 Acceptable"
        else:
            yaw_rating = "🔴 Needs improvement"
        
        print(f"   • Heading tracking rating: {yaw_rating}")
    
    # 💡 INTERPRETATION SUMMARY
    print(f"\n💡 KEY FINDINGS:")
    print("-" * 15)
    print(f"✅ Position coordinates (error_x/y/z): {pos_magnitude.mean():.0f}±{pos_magnitude.std():.0f} m from origin")
    print(f"✅ True tracking error (net distance): {guidance_data['error_net_distance'].abs().mean():.3f} m average")
    print(f"✅ Velocity control performance: {np.mean(velocity_performance):.4f} m/s average error" if velocity_performance else "❌ No velocity error data")
    print(f"✅ Heading control performance: {np.degrees(guidance_data['error_yaw'].abs().mean()):.1f}° average error" if 'error_yaw' in guidance_data.columns else "❌ No yaw error data")
    
    # Scale issue warnings
    print(f"\n⚠️ SCALE VERIFICATION:")
    print(f"   • Position 'errors' are actually COORDINATES relative to reference frame")
    print(f"   • Angular values appear to be in RADIANS (verify with mission parameters)")
    print(f"   • True performance metrics are reasonably scaled for AUV operations")

else:
    print("❌ No guidance data available for analysis")

🎯 Guidance System Error Analysis
📁 Loaded guidance__2024-08-20_13-40-35_data.csv: 350 rows, 30 columns
🎯 Analyzing guidance errors for bag: 2024-08-20_13-40-35


📁 Loaded guidance__2024-08-20_13-40-35_data.csv: 350 rows, 30 columns

📊 Guidance Data Summary:
   📈 Total samples: 350
   ⏱️ Duration: 0.71 minutes
   📊 Sampling rate: 494.0 samples/min

🔍 SCALE & UNIT VERIFICATION:
-----------------------------------
📍 POSITION VALUES (These are coordinates, NOT errors):
   • error_x: Range=[-527.5, -523.0] m
     Mean=-524.1 m, Std=0.8 m
   • error_y: Range=[-94.1, -92.4] m
     Mean=-93.3 m, Std=0.4 m
   • error_z: Range=[-492.4, -486.9] m
     Mean=-487.9 m, Std=0.9 m
   • 3D Position Magnitude: Range=[720.8, 727.3] m
     (This represents distance from origin/reference frame)

🎯 TRUE TRACKING ERRORS (Properly scaled performance metrics):
   • Net_Distance: Mean=0.4042 m, Max=1.4000 m, RMS=0.5223 m
   • Surge: Mean=0.2429 m, Max=0.5560 m, RMS=0.2595 m
   • Sway: Mean=0.2945 m, Max=0.4401 m, RMS=0.2993 m
   • Heave: Mean=0.0281 m, Max=0.3152 m, RMS=0.0386 m
   • Depth: Mean=0.0000 m, Max=0.0000 m, RMS=0.0000 m ⚠️ VERY SMALL

🧭 ANGULAR ERRORS (Check

## 🧭 Navigation Plane Approximation Analysis

Analysis of the navigation plane approximation system including:
- Net distance and altitude tracking
- Velocity components (u, v, w)
- 3D position trajectory
- Heading and pitch dynamics

In [57]:
# 🧭 NAVIGATION PLANE APPROXIMATION ANALYSIS
# ==========================================

print("🧭 Navigation Plane Approximation Analysis")
print("=" * 45)

# Analyze navigation plane approximation
nav_analyzer.analyze_navigation_plane(selected_bag, interactive=HAS_PLOTLY)

# Load navigation data for detailed analysis
plane_data = nav_analyzer.load_sensor_data("navigation_plane_approximation", selected_bag)
plane_pos_data = nav_analyzer.load_sensor_data("navigation_plane_approximation_position", selected_bag)

if plane_data is not None:
    print(f"\n📊 Navigation Plane Data Summary:")
    print(f"   📈 Plane samples: {len(plane_data)}")
    print(f"   📍 Position samples: {len(plane_pos_data) if plane_pos_data is not None else 0}")
    print(f"   ⏱️ Duration: {plane_data['t_rel'].max()/60:.2f} minutes")
    print(f"   📊 Sampling rate: {len(plane_data)/(plane_data['t_rel'].max()/60):.1f} samples/min")
    
    # Navigation performance metrics
    if 'NetDistance' in plane_data.columns:
        net_dist_stats = plane_data['NetDistance'].describe()
        print(f"\n🧭 Net Distance Statistics:")
        print(f"   • Range: {net_dist_stats['min']:.3f} to {net_dist_stats['max']:.3f} m")
        print(f"   • Mean: {net_dist_stats['mean']:.3f} m")
        print(f"   • Std: {net_dist_stats['std']:.3f} m")
    
    if 'Altitude' in plane_data.columns:
        alt_stats = plane_data['Altitude'].describe()
        print(f"\n📏 Altitude Statistics:")
        print(f"   • Range: {alt_stats['min']:.3f} to {alt_stats['max']:.3f} m")
        print(f"   • Mean: {alt_stats['mean']:.3f} m")
        print(f"   • Std: {alt_stats['std']:.3f} m")
    
    # Velocity analysis
    vel_cols = ['NetVelocity_u', 'NetVelocity_v', 'NetVelocity_w']
    print(f"\n🚀 Velocity Analysis:")
    for col in vel_cols:
        if col in plane_data.columns:
            vel_stats = plane_data[col].describe()
            print(f"   • {col}: Mean={vel_stats['mean']:.3f} m/s, Max={vel_stats['max']:.3f} m/s")

else:
    print("❌ No navigation plane data available for analysis")

🧭 Navigation Plane Approximation Analysis
📁 Loaded navigation_plane_approximation__2024-08-20_13-40-35_data.csv: 374 rows, 23 columns
📁 Loaded navigation_plane_approximation_position__2024-08-20_13-40-35_data.csv: 196 rows, 22 columns
🧭 Analyzing navigation plane for bag: 2024-08-20_13-40-35


📁 Loaded navigation_plane_approximation__2024-08-20_13-40-35_data.csv: 374 rows, 23 columns
📁 Loaded navigation_plane_approximation_position__2024-08-20_13-40-35_data.csv: 196 rows, 22 columns

📊 Navigation Plane Data Summary:
   📈 Plane samples: 374
   📍 Position samples: 196
   ⏱️ Duration: 0.78 minutes
   📊 Sampling rate: 481.4 samples/min

🧭 Net Distance Statistics:
   • Range: 0.540 to 3.180 m
   • Mean: 1.445 m
   • Std: 0.483 m

📏 Altitude Statistics:
   • Range: -1.000 to 3.412 m
   • Mean: 1.685 m
   • Std: 0.421 m

🚀 Velocity Analysis:
   • NetVelocity_u: Mean=0.015 m/s, Max=0.411 m/s
   • NetVelocity_v: Mean=-0.006 m/s, Max=0.140 m/s
   • NetVelocity_w: Mean=-0.001 m/s, Max=0.315 m/s


## ⚖️ Navigation vs Guidance System Comparison

Comparative analysis between navigation and guidance systems:
- Position tracking comparison
- Velocity performance comparison
- Error correlation analysis
- System synchronization assessment

In [58]:
# ⚖️ NAVIGATION vs GUIDANCE COMPARISON
# ===================================

print("⚖️ Navigation vs Guidance System Comparison")
print("=" * 45)

# Compare navigation and guidance systems
nav_analyzer.compare_navigation_guidance(selected_bag, interactive=HAS_PLOTLY)

if guidance_data is not None and plane_data is not None:
    print(f"\n📊 System Comparison Summary:")
    
    # Data coverage comparison
    guidance_duration = guidance_data['t_rel'].max() / 60.0
    plane_duration = plane_data['t_rel'].max() / 60.0
    
    print(f"   📈 Data Coverage:")
    print(f"     • Guidance: {len(guidance_data)} samples over {guidance_duration:.2f} min")
    print(f"     • Navigation: {len(plane_data)} samples over {plane_duration:.2f} min")
    
    # Sampling rate comparison
    guidance_rate = len(guidance_data) / guidance_duration
    nav_rate = len(plane_data) / plane_duration
    rate_diff = abs(guidance_rate - nav_rate)
    
    print(f"   📊 Sampling Rates:")
    print(f"     • Guidance: {guidance_rate:.1f} samples/min")
    print(f"     • Navigation: {nav_rate:.1f} samples/min")
    print(f"     • Rate difference: {rate_diff:.1f} samples/min")
    
    # Time synchronization check
    guidance_time_gaps = np.diff(guidance_data['t_rel'])
    plane_time_gaps = np.diff(plane_data['t_rel'])
    
    print(f"   ⏱️ Time Synchronization:")
    print(f"     • Guidance max gap: {guidance_time_gaps.max():.2f} seconds")
    print(f"     • Navigation max gap: {plane_time_gaps.max():.2f} seconds")
    print(f"     • Sync quality: {'✅ Good' if rate_diff < 20 else '⚠️ Attention needed'}")

else:
    print("❌ Cannot perform comparison - missing required data")

⚖️ Navigation vs Guidance System Comparison
📁 Loaded guidance__2024-08-20_13-40-35_data.csv: 350 rows, 30 columns
📁 Loaded navigation_plane_approximation__2024-08-20_13-40-35_data.csv: 374 rows, 23 columns
📁 Loaded navigation_plane_approximation_position__2024-08-20_13-40-35_data.csv: 196 rows, 22 columns
⚖️  Comparing navigation and guidance for bag: 2024-08-20_13-40-35

📊 Data Summary:
  - Guidance samples: 350
  - Navigation plane samples: 374
  - Navigation position samples: 196

🕐 Sampling Rates:
  - Guidance: 494.0 samples/min
  - Navigation plane: 481.4 samples/min



📊 System Comparison Summary:
   📈 Data Coverage:
     • Guidance: 350 samples over 0.71 min
     • Navigation: 374 samples over 0.78 min
   📊 Sampling Rates:
     • Guidance: 494.0 samples/min
     • Navigation: 481.4 samples/min
     • Rate difference: 12.6 samples/min
   ⏱️ Time Synchronization:
     • Guidance max gap: 1.10 seconds
     • Navigation max gap: 1.10 seconds
     • Sync quality: ✅ Good


## 📍 Position Trajectory Analysis

Detailed analysis of the vehicle's position trajectory:
- 3D path visualization
- Distance calculations
- Mission profile assessment
- Path efficiency metrics

In [59]:
# 📍 POSITION TRAJECTORY ANALYSIS
# ===============================

print("📍 Position Trajectory Analysis")
print("=" * 35)

if plane_pos_data is not None and all(col in plane_pos_data.columns for col in ['x', 'y', 'z']):
    
    # Basic trajectory statistics
    print(f"\n📊 Trajectory Statistics:")
    x_range = plane_pos_data['x'].max() - plane_pos_data['x'].min()
    y_range = plane_pos_data['y'].max() - plane_pos_data['y'].min()
    z_range = plane_pos_data['z'].max() - plane_pos_data['z'].min()
    
    print(f"   • X Range: {x_range:.2f} m")
    print(f"   • Y Range: {y_range:.2f} m")
    print(f"   • Z Range: {z_range:.2f} m")
    
    # Calculate trajectory metrics
    dx = np.diff(plane_pos_data['x'])
    dy = np.diff(plane_pos_data['y'])
    dz = np.diff(plane_pos_data['z'])
    
    # Segment distances
    distances = np.sqrt(dx**2 + dy**2 + dz**2)
    total_distance = np.sum(distances)
    
    # Direct distance (start to end)
    start_pos = np.array([plane_pos_data['x'].iloc[0], plane_pos_data['y'].iloc[0], plane_pos_data['z'].iloc[0]])
    end_pos = np.array([plane_pos_data['x'].iloc[-1], plane_pos_data['y'].iloc[-1], plane_pos_data['z'].iloc[-1]])
    direct_distance = np.linalg.norm(end_pos - start_pos)
    
    print(f"\n🛤️ Path Metrics:")
    print(f"   • Total path length: {total_distance:.2f} m")
    print(f"   • Direct distance: {direct_distance:.2f} m")
    print(f"   • Path efficiency: {(direct_distance/total_distance)*100:.1f}%")
    print(f"   • Average segment length: {distances.mean():.3f} m")
    print(f"   • Maximum segment length: {distances.max():.3f} m")
    
    # Mission type classification
    efficiency = direct_distance / total_distance
    if efficiency > 0.8:
        mission_type = "🎯 Point-to-point navigation"
    elif efficiency < 0.3:
        mission_type = "🔄 Survey/patrol pattern"
    else:
        mission_type = "🛤️ Structured path following"
    
    print(f"   • Mission classification: {mission_type}")
    
    # Create 2D trajectory plot
    if HAS_PLOTLY:
        fig = go.Figure()
        
        # Add trajectory
        fig.add_trace(go.Scatter(
            x=plane_pos_data['x'], 
            y=plane_pos_data['y'],
            mode='lines+markers',
            name='Trajectory',
            line=dict(color='blue', width=2),
            marker=dict(size=4)
        ))
        
        # Mark start and end points
        fig.add_trace(go.Scatter(
            x=[plane_pos_data['x'].iloc[0]], 
            y=[plane_pos_data['y'].iloc[0]],
            mode='markers',
            name='Start',
            marker=dict(color='green', size=12, symbol='circle')
        ))
        
        fig.add_trace(go.Scatter(
            x=[plane_pos_data['x'].iloc[-1]], 
            y=[plane_pos_data['y'].iloc[-1]],
            mode='markers',
            name='End',
            marker=dict(color='red', size=12, symbol='x')
        ))
        
        fig.update_layout(
            title="2D Position Trajectory",
            xaxis_title="X Position (m)",
            yaxis_title="Y Position (m)",
            showlegend=True,
            width=800,
            height=600
        )
        
        fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Equal aspect ratio
        fig.show()
    
    else:
        # Matplotlib fallback
        plt.figure(figsize=(10, 8))
        plt.plot(plane_pos_data['x'], plane_pos_data['y'], 'b-', alpha=0.7, linewidth=2, label='Trajectory')
        plt.scatter(plane_pos_data['x'].iloc[0], plane_pos_data['y'].iloc[0], 
                   color='green', s=100, label='Start', zorder=5)
        plt.scatter(plane_pos_data['x'].iloc[-1], plane_pos_data['y'].iloc[-1], 
                   color='red', s=100, label='End', zorder=5, marker='x')
        
        plt.title('2D Position Trajectory')
        plt.xlabel('X Position (m)')
        plt.ylabel('Y Position (m)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.axis('equal')
        plt.show()

else:
    print("❌ No position data available for trajectory analysis")

📍 Position Trajectory Analysis

📊 Trajectory Statistics:
   • X Range: 5.32 m
   • Y Range: 8.04 m
   • Z Range: 14.06 m

🛤️ Path Metrics:
   • Total path length: 45.96 m
   • Direct distance: 15.32 m
   • Path efficiency: 33.3%
   • Average segment length: 0.236 m
   • Maximum segment length: 2.983 m
   • Mission classification: 🛤️ Structured path following
